In [1]:
import sys
import pandas as pd
import numpy as np
import xarray as xr         # มันคือ pandas สำหรับข้อมูล 3 มิติ เช่น NetCDF (Time, Lat, Long)
import rioxarray            # ส่วนเสริมที่ทำให้ xarray เข้าใจไฟล์ภาพ GeoTIFF
from tqdm import tqdm
from pathlib import Path    
from collections import defaultdict

ROOT_PATH = Path.cwd().parent
if ROOT_PATH not in sys.path:
    sys.path.append(str(ROOT_PATH)) 
    
DATA_PATH = ROOT_PATH / "data" 
RAIN_OBS = DATA_PATH / "raw" / "rain_obs_monthly"
PROCESSED_DATA_PATH = DATA_PATH / "processed" 

RAIN_OBS.mkdir(parents=True, exist_ok=True)
PROCESSED_DATA_PATH.mkdir(parents=True, exist_ok=True)

In [2]:
# 1. เตรียม Dictionary {"yyyymm": Path("...")} เช่น {"197001": "Path("o_th197001.tif"), ..."}
# 2. กวาดหาไฟล์
# 3. คัดเลือกไฟล์
# 4. ส่งออก output

rain_files = RAIN_OBS.glob("o_th*.tif")
rain_dict = {}

for p in rain_files:
        month_key = p.stem.replace("_fixed", "")\
            
        is_fixed = "_fixed" in p.stem
        
        if month_key not in rain_dict:
            rain_dict[month_key] = p
        elif is_fixed:
            rain_dict[month_key] = p
    
rain_paths = sorted(list(rain_dict.values()))

rain_paths[-6:-1]

[WindowsPath('d:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th202507.tif'),
 WindowsPath('d:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th202508.tif'),
 WindowsPath('d:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th202509.tif'),
 WindowsPath('d:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th202510.tif'),
 WindowsPath('d:/HII/drought_forecasting/data/raw/rain_obs_monthly/o_th202511.tif')]

In [ ]:
rain_da_list = []    # da ย่อมาจาก DataArray (xarray.DataArray)

for path in tqdm(rain_paths):
    dt = pd.to_datetime(path.stem[4:10], format="%Y%m")

    # masked=True เปลี่ยนค่า nodata ให้เป็น NaN
    with rioxarray.open_rasterio(path, masked=True, chunks="auto") as rain_file:
        da = rain_file.squeeze(dim="band", drop=True).expand_dims(time=[dt])
        
        da.name = "precipitation"
        rain_da_list.append(da)

combined_rain_da = xr.concat(rain_da_list, dim='time', join='override')    # rain_ds ย่อมาจาก Dataset (xaray.Dataset)

# ล้าง Attributes ที่ไม่จำเป็นทิ้ง
keys_to_remove = ['STATISTICS_MAXIMUM', 'STATISTICS_MINIMUM', 
                  'STATISTICS_MEAN', 'STATISTICS_STDDEV']

for key in keys_to_remove:
    if key in combined_rain_da.attrs:
        del combined_rain_da.attrs[key]

rain_ds = combined_rain_da.to_dataset()



100%|██████████| 672/672 [00:05<00:00, 112.65it/s]


In [ ]:
rain_ds

<xarray.Dataset> Size: 3GB
Dimensions:        (time: 672, x: 829, y: 1485)
Coordinates:
  * time           (time) datetime64[us] 5kB 1970-01-01 ... 2025-12-01
  * x              (x) float64 7kB 97.35 97.36 97.37 97.38 ... 105.6 105.6 105.6
  * y              (y) float64 12kB 20.46 20.45 20.44 20.43 ... 5.64 5.63 5.62
    spatial_ref    int64 8B 0
Data variables:
    precipitation  (time, y, x) float32 3GB dask.array<chunksize=(1, 1485, 829), meta=np.ndarray>

In [ ]:
rain_da_list

[<xarray.DataArray 'precipitation' (time: 1, y: 1485, x: 829)> Size: 5MB
 dask.array<getitem, shape=(1, 1485, 829), dtype=float32, chunksize=(1, 1485, 829), chunktype=numpy.ndarray>
 Coordinates:
   * time         (time) datetime64[us] 8B 1970-01-01
   * y            (y) float64 12kB 20.46 20.45 20.44 20.43 ... 5.64 5.63 5.62
   * x            (x) float64 7kB 97.35 97.36 97.37 97.38 ... 105.6 105.6 105.6
     spatial_ref  int64 8B 0
 Attributes:
     AREA_OR_POINT:       Area
     STATISTICS_MAXIMUM:  387.69
     STATISTICS_MEAN:     24.478840240287
     STATISTICS_MINIMUM:  0
     STATISTICS_STDDEV:   55.386335528189
     scale_factor:        1.0
     add_offset:          0.0,
 <xarray.DataArray 'precipitation' (time: 1, y: 1485, x: 829)> Size: 5MB
 dask.array<getitem, shape=(1, 1485, 829), dtype=float32, chunksize=(1, 1485, 829), chunktype=numpy.ndarray>
 Coordinates:
   * time         (time) datetime64[us] 8B 1970-02-01
   * y            (y) float64 12kB 20.46 20.45 20.44 20.43 ... 

In [6]:
temp = rioxarray.open_rasterio(rain_paths[0])
temp  # หรือ print(temp)

<xarray.DataArray (band: 1, y: 1485, x: 829)> Size: 5MB
[1231065 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * y            (y) float64 12kB 20.46 20.45 20.44 20.43 ... 5.64 5.63 5.62
  * x            (x) float64 7kB 97.35 97.36 97.37 97.38 ... 105.6 105.6 105.6
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  387.69
    STATISTICS_MEAN:     24.478840240287
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   55.386335528189
    _FillValue:          -3.4e+38
    scale_factor:        1.0
    add_offset:          0.0

In [7]:
import dask
print(f"Dask Version: {dask.__version__}")

Dask Version: 2026.1.2


In [8]:
temp.dims

('band', 'y', 'x')

In [9]:
temp2 = temp.squeeze(dim="band")
temp2

<xarray.DataArray (y: 1485, x: 829)> Size: 5MB
[1231065 values with dtype=float32]
Coordinates:
  * y            (y) float64 12kB 20.46 20.45 20.44 20.43 ... 5.64 5.63 5.62
  * x            (x) float64 7kB 97.35 97.36 97.37 97.38 ... 105.6 105.6 105.6
    band         int64 8B 1
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  387.69
    STATISTICS_MEAN:     24.478840240287
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   55.386335528189
    _FillValue:          -3.4e+38
    scale_factor:        1.0
    add_offset:          0.0

In [10]:
temp2.dims

('y', 'x')